# Advent of Code

## 2024-012-009
## 2024 009

https://adventofcode.com/2024/day/9

In [1]:
def main():
    # Read the disk map from input.txt
    with open("input.txt", "r") as f:
        disk_map = f.read().strip()

    # Parse the disk map into a sequence of (file_length, free_length) pairs.
    # Actually, it’s interleaved: even indices are file lengths, odd indices are free lengths.
    lengths = list(map(int, disk_map))
    
    # Build the disk array
    # We'll have something like: [file0]*f0_length + ['.']*free_length + [file1]*f1_length + ...
    disk = []
    file_id = 0
    for i, length in enumerate(lengths):
        if i % 2 == 0:
            # file length
            if length > 0:
                disk.extend([file_id] * length)
            # Move to next file id after placing a file
            file_id += 1
        else:
            # free space length
            if length > 0:
                disk.extend(['.'] * length)

    # The last segment (if the number of digits is odd) would end with a file length, 
    # and if even length of digits, it ends in free space. The problem statement's input 
    # suggests a well-formed disk map always alternating. We'll assume it's correct.

    # Now we must compact:
    # The method: 
    # While there's a '.' in the disk between file blocks, move one file block
    # from the right end into the leftmost '.'.
    # In other words:
    # Find leftmost '.', find the rightmost file block, move it.

    # An efficient approach:
    # We know every '.' must be filled by a file block from the end.
    # We can:
    # 1) Count how many '.' are in total before the final contiguous block of files at the end.
    # 2) We'll repeatedly pop from the end until all '.' are replaced.

    # However, we must follow the exact procedure described:
    # The procedure implies every '.' will be replaced by one of the last file blocks.

    # We'll just simulate directly:
    from collections import deque

    disk_deque = deque(disk)

    # While there exists a '.' in disk_deque[:-1], fill it:
    # Actually, we must ensure no '.' remains between file blocks.
    # After compaction, all '.' should be at the far right end (or not exist at all).

    # The final state after compaction is that all '.' are moved to the end of the disk (or gone),
    # but the instructions say we move blocks from the end to the leftmost '.' until no '.' is left among files.
    # Eventually, this process should consume all '.' in between.

    # We'll do:
    # - Find the leftmost '.' each iteration.
    # - Pop from the right until we get a file block.
    # - Place that file block in the '.' position.
    # Repeat until no '.' is left in any position before the last file block.

    # A direct simulation might be slow for extremely large input.
    # But we'll trust a well-optimized Python approach should handle it.

    # Let's implement the direct simulation:
    while True:
        try:
            gap_index = disk_deque.index('.')  # leftmost '.' index
        except ValueError:
            # No '.' found
            break

        # We have a '.' at gap_index
        # Move from the right until we find a file block
        # Since trailing blocks might be files or '.' we must pop from the right until we get a file ID
        right_block = disk_deque.pop()
        while right_block == '.':
            # If we popped '.', put it aside
            # Actually, if we pop '.', these are trailing '.' at the end, which means 
            # we can just ignore them because eventually they need not remain in the middle.
            # But let's store them to re-append after we find a file block.
            # Although the instructions suggest we always move a file block, 
            # trailing '.' means we have free space at the end beyond all files.
            # We must keep popping until we get a file block.
            right_block = disk_deque.pop()

        # Now right_block is a file block (an integer)
        # Place that block into the '.' position
        disk_deque[gap_index] = right_block

    # After compaction, compute checksum:
    checksum = 0
    for i, block in enumerate(disk_deque):
        if block != '.':
            # block is a file ID (integer)
            checksum += i * block

    print(checksum)

if __name__ == "__main__":
    main()

6258319840548


In [2]:
def main():
    # Read input
    with open("input.txt", "r") as f:
        disk_map = f.read().strip()

    # Parse the disk map
    lengths = list(map(int, disk_map))

    disk = []
    file_id = 0
    for i, length in enumerate(lengths):
        if i % 2 == 0:
            # file length
            if length > 0:
                disk.extend([file_id] * length)
            file_id += 1
        else:
            # free space
            if length > 0:
                disk.extend(['.'] * length)

    # The number of files is half the length of `lengths` if even, 
    # or (len(lengths)+1)//2, but since we increment file_id for every file length, 
    # the total number of files = the last file_id assigned.
    total_files = file_id

    # Gather file info: start, end, length
    # files[file_id] = {"start": s, "end": e, "length": l}
    files = {}
    current_id = None
    current_start = None
    for i, block in enumerate(disk):
        if block != '.':
            b_id = block
            if b_id not in files:
                files[b_id] = {"start": i, "end": i, "length": 1}
            else:
                files[b_id]["end"] = i
                files[b_id]["length"] += 1

    # We must move files in decreasing order of ID
    # For each file, try to find a free segment to the left of its start that fits the file
    # Procedure:
    # 1) Identify file start, length
    # 2) Scan free space segments from left to right up to the file's start position
    # 3) If found a suitable segment, move the file there

    # Let's define a helper to find a suitable segment:
    def find_segment_for_file(file_length, max_index):
        # Find a free space segment in disk[0:max_index] that can fit file_length
        # Return the start index of that segment or None
        count = 0
        segment_start = None
        for i in range(max_index):
            if disk[i] == '.':
                if segment_start is None:
                    segment_start = i
                count += 1
                if count == file_length:
                    # Found a suitable segment
                    return segment_start
            else:
                # Reset count
                count = 0
                segment_start = None
        return None

    # Move files in decreasing order
    for f_id in sorted(files.keys(), reverse=True):
        f_info = files[f_id]
        f_length = f_info["length"]
        f_start = f_info["start"]
        f_end = f_info["end"]

        # Find a suitable segment for this file to the left of f_start
        target_start = find_segment_for_file(f_length, f_start)
        if target_start is not None:
            # Move the file
            # Clear old positions
            for pos in range(f_start, f_end+1):
                disk[pos] = '.'
            # Fill new positions
            for pos in range(target_start, target_start+f_length):
                disk[pos] = f_id
            # Update file info
            new_start = target_start
            new_end = target_start + f_length - 1
            files[f_id]["start"] = new_start
            files[f_id]["end"] = new_end
            # No need to update length since it doesn't change

    # Compute checksum
    checksum = 0
    for i, block in enumerate(disk):
        if block != '.':
            checksum += i * block

    print(checksum)


if __name__ == "__main__":
    main()

6286182965311


In [2]:
import time
from collections import deque

def parse_disk_map(disk_map):
    """Parses the disk map into a list of blocks."""
    lengths = list(map(int, disk_map))
    disk = []
    file_id = 0
    for i, length in enumerate(lengths):
        if i % 2 == 0:
            if length > 0:
                disk.extend([file_id] * length)
            file_id += 1
        else:
            if length > 0:
                disk.extend(['.'] * length)
    return disk

def compact_blocks(disk):
    """Simulates moving individual blocks to compact the disk."""
    disk_deque = deque(disk)
    while True:
        try:
            gap_index = disk_deque.index('.')
        except ValueError:
            break
        right_block = disk_deque.pop()
        while right_block == '.':
            right_block = disk_deque.pop()
        disk_deque[gap_index] = right_block
    return list(disk_deque)

def compact_files(disk, files):
    """Simulates moving entire files to compact the disk."""
    def find_segment_for_file(file_length, max_index):
        count = 0
        segment_start = None
        for i in range(max_index):
            if disk[i] == '.':
                if segment_start is None:
                    segment_start = i
                count += 1
                if count == file_length:
                    return segment_start
            else:
                count = 0
                segment_start = None
        return None

    for f_id in sorted(files.keys(), reverse=True):
        f_info = files[f_id]
        f_length = f_info["length"]
        f_start = f_info["start"]
        target_start = find_segment_for_file(f_length, f_start)
        if target_start is not None:
            for pos in range(f_start, f_start + f_length):
                disk[pos] = '.'
            for pos in range(target_start, target_start + f_length):
                disk[pos] = f_id
    return disk

def calculate_checksum(disk):
    """Calculates the checksum of the disk."""
    checksum = sum(i * block for i, block in enumerate(disk) if block != '.')
    return checksum

def get_file_info(disk):
    """Extracts information about each file on the disk."""
    files = {}
    for i, block in enumerate(disk):
        if block != '.':
            if block not in files:
                files[block] = {"start": i, "length": 0}
            files[block]["length"] += 1
    return files

def main():
    # Read the disk map from input.txt
    with open("input.txt", "r") as f:
        disk_map = f.read().strip()

    # Parse the disk map
    start_total = time.time()
    start_part1 = time.time()
    disk = parse_disk_map(disk_map)
    end_part1 = time.time()

    print(f"Part 1 disk parsing done in {end_part1 - start_part1:.9f} s")

    # Compact blocks (Part 1)
    start_compact_blocks = time.time()
    compacted_disk = compact_blocks(disk)
    part1_checksum = calculate_checksum(compacted_disk)
    end_compact_blocks = time.time()

    print(f"Part 1 answer is {part1_checksum}, found in {end_compact_blocks - start_compact_blocks:.9f} s")

    # Gather file information
    files = get_file_info(disk)

    # Compact files (Part 2)
    start_part2 = time.time()
    compacted_disk_files = compact_files(disk.copy(), files)
    part2_checksum = calculate_checksum(compacted_disk_files)
    end_part2 = time.time()

    print(f"Part 2 answer is {part2_checksum}, found in {end_part2 - start_part2:.9f} s")

    # Total time
    end_total = time.time()
    print(f"Total time was {end_total - start_total:.9f} s")

if __name__ == "__main__":
    main()

Part 1 disk parsing done in 0.007000446 s
Part 1 answer is 6258319840548, found in 20.837277651 s
Part 2 answer is 6286182965311, found in 27.187501431 s
Total time was 48.086116791 s
